# Installing Libraries




In [ ]:
!apt install libpoppler-cpp-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpoppler-cpp0v5
The following NEW packages will be installed:
  libpoppler-cpp-dev libpoppler-cpp0v5
0 upgraded, 2 newly installed, 0 to remove and 19 not upgraded.
Need to get 50.4 kB of archives.
After this operation, 231 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp0v5 amd64 22.02.0-2ubuntu0.2 [38.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libpoppler-cpp-dev amd64 22.02.0-2ubuntu0.2 [11.8 kB]
Fetched 50.4 kB in 0s (316 kB/s)
Selecting previously unselected package libpoppler-cpp0v5:amd64.
(Reading database ... 120874 files and directories currently installed.)
Preparing to unpack .../libpoppler-cpp0v5_22.02.0-2ubuntu0.2_amd64.deb ...
Unpacking libpoppler-cpp0v5:amd64 (22.02.0-2ubuntu0.2) ...
Selecting previously unselected package 

In [ ]:
!pip install pdftotext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pdftotext: filename=pdftotext-2.2.2-cp310-cp310-linux_x86_64.whl size=61846 sha256=4514cb3182a42a5e611af4e456aec621027b7678625446b7a6acb78e9b8e9b1d
  Stored in directory: /root/.cache/pip/wheels/c8/68/c6/82a8c2046e1a8f9051017319dc007f43ef3a6edc53149d1f4e
Successfully built pdftotext


In [ ]:
!pip install transformers
!python -m nltk.downloader punkt
!pip install sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 47.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 103.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 24.4 MB/s eta 0:00:00
/usr/lib/python3.10/runpy.py:126: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.8 MB/s eta 0:00:00


In [ ]:
pip install nltk

# Preprocessing

In [ ]:
import pdftotext
import re

class TextProcessingAndContextCreation :

    @classmethod
    def get_context_chunks(cls, file_name) :

        raw_text = cls._get_raw_text_from_pdf(file_name)
        first_page_number = cls._get_the_first_page_number(raw_text)
        if first_page_number == -1 :
            print("Error encountered")
            return []
        raw_text = raw_text[first_page_number:]
        cls._remove_header_of_first_page(raw_text)
        cls._delete_footer_notes(raw_text)
        cls._remove_captial_words(raw_text)
        text = cls._join_all_pages(raw_text)
        text = cls._split_the_text_on_bold_points(text)
        cls._filter_and_get_plain_text(text)
        # return text
        contexts =  cls._get_the_contexts(text)
        return contexts

    @classmethod
    def _get_raw_text_from_pdf(cls, file_name) :

        with open(file_name, "rb") as f :
            pdf_text = pdftotext.PDF(f)

        raw_text = []
        for i in pdf_text :
            raw_text.append(str(i))
        return raw_text

    @classmethod
    def _get_the_first_page_number(cls, raw_text) :

        index = raw_text[0].split().index('ACT,')
        match_text_page_zero = " ".join(raw_text[0].split()[:index + 1])

        for i in range(1, len(raw_text)) :
            page_match_text = " ".join(raw_text[i].split()[:index + 1])
            if match_text_page_zero == page_match_text :
                return i
        return -1

    @classmethod
    def _remove_header_of_first_page(cls, raw_text) :

        temp = re.match(r"(.|\n)*?", raw_text[0])
        raw_text[0] = raw_text[0][temp.end():]

    @classmethod
    def _delete_footer_notes(cls, raw_text) :

        footer_regex = r"1\..*"
        for i in range(len(raw_text)) :
            temp = raw_text[i].split('\n')[:-2]
            for j in range(len(temp) - 1, -1, -1) :
                if re.match(footer_regex, temp[j]) is not None :
                    break
            if j != 0 :
                temp = temp[:j]
            raw_text[i] = "\n".join(temp)

    @classmethod
    def _remove_captial_words(cls, raw_text) :

        for i in range(len(raw_text)) :
            raw_text[i] = raw_text[i].split('\n')
            raw_text[i] = list(filter(lambda x: not (x.isupper()), raw_text[i]))
            raw_text[i] = "\n".join(raw_text[i])

    @classmethod
    def _join_all_pages(cls, raw_text) :

        concatenated_text = []
        for i in raw_text :
            concatenated_text.append(i)
        concatenated_text = "\n".join(concatenated_text)
        return concatenated_text

    @classmethod
    def _split_the_text_on_bold_points(cls, text) :

        split_regex = r"[0-9]+[A-Z]*\.[^\n]"
        split_text = re.split(split_regex, text)
        return split_text

    @classmethod
    def _filter_and_get_plain_text(cls, text) :

        for i in range(len(text)) :
            text[i] = re.sub(r"(“|”|’)", "\"", text[i])
            text[i] = re.sub(r"\n", " ", text[i])
            text[i] = re.sub(r" +", " ", text[i])
            text[i] = re.sub(r"—", "- ", text[i])

    @classmethod
    def _get_the_contexts(cls, text) :

        contexts = []
        for i in range(len(text)) :
            if (len(text[i].split()) > 10 and len(text[i].split()) <= 512) :
                contexts.append(text[i])
            else :
                temp = cls._get_contexts_greater_than_max_size(text[i])
                for i in temp :
                    contexts.append(" ".join(i))

        return contexts

    @classmethod
    def _get_contexts_greater_than_max_size(cls, text) :

        ROMAN_NUMERALS = ['i', 'ii', 'iii', 'iv', 'v', 'vi', 'vii', 'viii', 'ix', 'x', \
                          'xi', 'xii', 'xiii', 'xiv', 'xv', 'xvi', 'xvii', 'xviii', 'xix', 'xx', \
                          'xxi', 'xxii', 'xxiii', 'xxiv', 'xxv', 'xxvi', 'xxvii', 'xxviii', 'xxix', 'xxx']
        level_wise_regex = [r"[0-9]+", r"[a-z]", r"\((" + "|".join(ROMAN_NUMERALS) + r")\)" , r"[A-Z]"]


        punctuations = ",-.:;"
        end_markers = "."
        current_index = 0
        regex_match_flag = 0
        max_context_length = 0

        contexts = []
        text = text.split()

        while (current_index < len(text)) :

            regex_match_flag = 0
            max_context_length = min(current_index + 512, len(text))
            context = text[current_index:max_context_length]
            if max_context_length == len(text) :
                contexts.append(context)
                current_index += len(context)
            else :
                i = len(context) - 1
                while i > (len(context) // 2) :
                    if (re.match(level_wise_regex[0], context[i]) is not None) :
                        print("First Matched")
                    if (re.match(level_wise_regex[1], context[i]) is not None) :
                        print("Second Matched")
                    if (re.match(level_wise_regex[2], context[i]) is not None) :
                        print("Third Matched")
                    if (re.match(level_wise_regex[3], context[i]) is not None) :
                        print("Fourth Matched")

                    # if (re.match(level_wise_regex[0], context[i]) is not None) or \
                    #     (re.match(level_wise_regex[1], context[i]) is not None) or \
                    #     (re.match(level_wise_regex[2], context[i]) is not None) or \
                    #     (re.match(level_wise_regex[3], context[i]) is not None) :
                    #     if i != 0 and context[i-1][-1] in punctuations :
                    #         context = context[:i]
                    #         contexts.append(context)
                    #         current_index += i
                    #         regex_match_flag = 1
                    #         break
                    # i -= 1

                if not regex_match_flag :
                    i = len(context) - 1
                    while (i > 0) :
                        if context[i][-1] in end_markers :
                            context = context[:i + 1]
                            contexts.append(context)
                            current_index += (i + 1)
                            break
                        i -= 1

            if len(contexts[-1]) <= 10 :
                del contexts[-1]

        return contexts

In [ ]:
legal_doc="/content/drive/MyDrive/Colab Notebooks/NLP PROJECT/legal_docs/A1897-03.pdf"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
contexts2=TextProcessingAndContextCreation.get_context_chunks(legal_doc)

In [ ]:
contexts2

[' An Act to provide for the better prevention of the spread of Dangerous Epidemic Diseases. WHEREAS it is expedient to provide for the better prevention of the spread of dangerous epidemic disease; It is hereby enacted as follows :-  ',
 'Short title and extent.- (1) This Act may be called the Epidemic Diseases Act, 1897. 2 [(2) It extends to the whole of India 3***] 4* * * 5 * * * [',
 'Definitions.- In this, unless the context otherwise requires,-  * * 6 (a) "act of violence" includes any of the following acts committed by any person against a healthcare service personnel serving during an epidemic, which causes or may cause -  (i) harassment impacting the living or working conditions of such healthcare service personnel and preventing him from discharging his duties; (ii) harm, injury, hurt, intimidation or danger to the life of such healthcare service personnel, either within the premises of a clinical establishment or otherwise; (iii) obstruction or hindrance to such healthcare s

In [ ]:
pip install -U gensim

In [ ]:
text = " ".join(contexts2)

In [ ]:
import nltk
nltk.download('punkt')
tokens = nltk.sent_tokenize(text)
for t in tokens:
    print(t, "\n")

 An Act to provide for the better prevention of the spread of Dangerous Epidemic Diseases. 

WHEREAS it is expedient to provide for the better prevention of the spread of dangerous epidemic disease; It is hereby enacted as follows :-   Short title and extent.- (1) This Act may be called the Epidemic Diseases Act, 1897. 

2 [(2) It extends to the whole of India 3***] 4* * * 5 * * * [ Definitions.- In this, unless the context otherwise requires,-  * * 6 (a) "act of violence" includes any of the following acts committed by any person against a healthcare service personnel serving during an epidemic, which causes or may cause -  (i) harassment impacting the living or working conditions of such healthcare service personnel and preventing him from discharging his duties; (ii) harm, injury, hurt, intimidation or danger to the life of such healthcare service personnel, either within the premises of a clinical establishment or otherwise; (iii) obstruction or hindrance to such healthcare service

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import re
import gensim
from gensim.parsing.preprocessing import remove_stopwords

def clean_sentence(sentence, stopwords=False):
  sentence = sentence.lower().strip()
  sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
  if stopwords:
    sentence = remove_stopwords(sentence)
  return sentence

def get_cleaned_sentences(tokens, stopwords=False):
  cleaned_sentences = []
  for row in tokens:
    cleaned = clean_sentence(row, stopwords)
    cleaned_sentences.append(cleaned)
  return cleaned_sentences

In [ ]:
cleaned_sentences = get_cleaned_sentences(tokens, stopwords=True)
cleaned_sentences_with_stopwords = get_cleaned_sentences(tokens, stopwords=False)
print(cleaned_sentences)
print(cleaned_sentences_with_stopwords)

['act provide better prevention spread dangerous epidemic diseases', 'expedient provide better prevention spread dangerous epidemic disease enacted follows short title extent 1 act called epidemic diseases act 1897', '2 2 extends india 3 4 5 definitions context requires 6 act violence includes following acts committed person healthcare service personnel serving epidemic causes cause harassment impacting living working conditions healthcare service personnel preventing discharging duties ii harm injury hurt intimidation danger life healthcare service personnel premises clinical establishment iii obstruction hindrance healthcare service personnel discharge duties premises clinical establishment iv loss damage property documents custody relation healthcare service personnel b healthcare service personnel means person carrying duties relation epidemic related responsibilities come direct contact affected patients risk impacted suc h disease includes public clinical healthcare provider doct

#Cosine Similarity

converting sentences of document into vector and converting questions into vector and apply brute force method to find the cosine similarity between the sentance vector and question vector and find the min angle between them it mean find the max cosine value between them and sentence with max cosine value is the answer of the given question.

Steps:

Scan whole document and map with words as key and value as index (make dictionary).
Split document into sentences (dataframes) and then store each row into list.
Make vector of this list.
Ex: Document : Age(index : 1), means(2), age(3), at(4), last(5), birthday(6), Accident(7), sudden(8), unforeseen(9), and(10), involuntary(11), event(12), caused(13), by(14), external(15), visible(16), and(17), violent(18)

Suppose sentence is "Age means ?"

Sentence Vector : 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Similary make User Question vector (after removing stoping words like what, how, why etc.)

Iterate whole sentance vector and find the cosine similarity with user question vector and find the max value (if the value is high it means angle between them is minimum and properties between both vectors matches with max probability).

In [ ]:
import numpy
sentences = cleaned_sentences_with_stopwords
sentence_words = [[word for word in document.split()]
                  for document in sentences]

from gensim import corpora
dictionary = corpora.Dictionary(sentence_words)
for key, value in dictionary.items():
  print(key, ' : ', value)

0  :  act
1  :  an
2  :  better
3  :  dangerous
4  :  diseases
5  :  epidemic
6  :  for
7  :  of
8  :  prevention
9  :  provide
10  :  spread
11  :  the
12  :  to
13  :  1
14  :  1897
15  :  and
16  :  as
17  :  be
18  :  called
19  :  disease
20  :  enacted
21  :  expedient
22  :  extent
23  :  follows
24  :  hereby
25  :  is
26  :  it
27  :  may
28  :  short
29  :  this
30  :  title
31  :  whereas
32  :  15
33  :  1908
34  :  1934
35  :  2
36  :  2010
37  :  22
38  :  23
39  :  3
40  :  31
41  :  4
42  :  5
43  :  6
44  :  a
45  :  acts
46  :  affected
47  :  against
48  :  aircraft
49  :  any
50  :  assign
51  :  at
52  :  authority
53  :  b
54  :  being
55  :  but
56  :  by
57  :  c
58  :  carrying
59  :  case
60  :  cause
61  :  causes
62  :  clinical
63  :  come
64  :  committed
65  :  community
66  :  conditions
67  :  contact
68  :  context
69  :  custody
70  :  d
71  :  damage
72  :  danger
73  :  declared
74  :  defined
75  :  definitions
76  :  direct
77  :  discharge
78  : 

In [ ]:
import pprint
bow_corpus = [dictionary.doc2bow(text) for text in sentence_words]
for sent, embedding in zip(sentences, bow_corpus):
  print(sent)
  print(embedding)

an act to provide for the better prevention of the spread of dangerous epidemic diseases
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1)]
whereas it is expedient to provide for the better prevention of the spread of dangerous epidemic disease it is hereby enacted as follows    short title and extent 1 this act may be called the epidemic diseases act 1897
[(0, 2), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 3), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 2), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1)]
2 2 it extends to the whole of india 3 4   5     definitions in this unless the context otherwise requires    6 a act of violence includes any of the following acts committed by any person against a healthcare service personnel serving during an epidemic which causes or may cause   i harassment impacting the liv

In [ ]:
question = "What does Composition of certain offences?"

In [ ]:
# Preprocessing of user question

In [ ]:
# import re
# import gensim
# from gensim.parsing.preprocessing import remove_stopwords

# def preprocess_question(question):
#     # Convert to lowercase and remove extra whitespace
#     question = question.lower().strip()
#     # Remove special characters and digits
#     question = re.sub(r'[^a-z\s]', '', question)
#     # Remove stopwords
#     question = remove_stopwords(question)
#     # Remove extra whitespace
#     question = re.sub(r'\s+', ' ', question).strip()
#     # Remove punctuation marks
#     question = re.sub(r'[.,!?]', '', question)
#     return question


# # Preprocess the question
# pre_question = preprocess_question(question)

# # Now 'preprocessed_question' contains the preprocessed question
# print("Original Question:", question)
# print("Preprocessed Question:", pre_question)


Original Question: what does composition of certain offences
Preprocessed Question: composition certain offences


In [ ]:
question = clean_sentence(question, stopwords=False)
question_embedding = dictionary.doc2bow(question.split())
print(question, "\n", question_embedding)

what does composition of certain offences 
 [(7, 1), (225, 1), (354, 1), (380, 1), (381, 1)]


In [ ]:
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

def retrieveAndPrintFAQAnswer(question_embedding, sentence_embeddings, sentences):
  max_sim = -1
  index_sim = -1
  for index, embedding in enumerate(sentence_embeddings):
    sim = cosine_similarity(embedding, question_embedding)[0][0]
    print(index, sim, sentences[index])
    if sim > max_sim:
      max_sim = sim
      index_sim = index

  return index_sim
index = retrieveAndPrintFAQAnswer(question_embedding, bow_corpus, sentences)

0 0.1414213562373095 an act to provide for the better prevention of the spread of dangerous epidemic diseases
1 0.1414213562373095 whereas it is expedient to provide for the better prevention of the spread of dangerous epidemic disease it is hereby enacted as follows    short title and extent 1 this act may be called the epidemic diseases act 1897
2 0.1414213562373095 2 2 it extends to the whole of india 3 4   5     definitions in this unless the context otherwise requires    6 a act of violence includes any of the following acts committed by any person against a healthcare service personnel serving during an epidemic which causes or may cause   i harassment impacting the living or working conditions of such healthcare service personnel and preventing him from discharging his duties ii harm injury hurt intimidation or danger to the life of such healthcare service personnel either within the premises of a clinical establishment or otherwise iii obstruction or hindrance to such healthcar

In [ ]:
print("Question: ", question)
print("Answer: ", sentences[index])

Question:  what does composition of certain offences
Answer:  explanation in this section culpable mental state includes intention motive knowledge of a fact and the belief in or reason to believe a fact


# Word2Vec



In these appproch we takes input of large corpus words and produces a vector space, typically of several hundreds dimensions, with each unique word in the corpus being assigned a corresponding vector in the space.

Word vectors are positioned in the vector space such that words that share common contexts in the corpus are located in close proximity to one another in the space. Word2Vec is a particularly computationally-efficient predictive model for learning word embeddings from raw text.

Working

Word2Vec is a simple neural network with a single hidden layer, and like all neural networks, it has weights, and during training, its goal is to adjust those weights to reduce a loss function. However, Word2Vec is not going to be used for the task it was trained on, instead, we will just take its hidden weights, use them as our word embeddings, and toss the rest of the model (this came out a little bit evil).


In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api

v2w_model = None
try:
  v2w_model = gensim.models.Keyedvectors.load('./w2vecmodel.mod')
  print("w2v Model Successfully loaded")
except:
  v2w_model = api.load('word2vec-google-news-300')
  v2w_model.save("./w2vecmodel.mod")
  print("w2v Model Saved")

w2vec_embedding_size = len(v2w_model['pc'])

w2v Model Saved


In [ ]:

def getWordVec(word, model):
  samp = model['pc']
  vec = [0]*len(samp)
  try:
    vec = model[word]
  except:
    vec = [0]*len(samp)
  return (vec)


def getPhraseEmbedding(phrase, embeddingmodel):
  samp = getWordVec('computer', embeddingmodel)
  vec = numpy.array([0]*len(samp))
  den = 0;
  for word in phrase.split():
    den = den+1
    vec = vec+numpy.array(getWordVec(word, embeddingmodel))
  return vec.reshape(1, -1)

In [ ]:
sent_embeddings = []
for sent in sentences:
  sent_embeddings.append(getPhraseEmbedding(sent, v2w_model))

question_embedding = getPhraseEmbedding(question, v2w_model)
index = retrieveAndPrintFAQAnswer(question_embedding, sent_embeddings, cleaned_sentences_with_stopwords)

0 0.3482360077532799 an act to provide for the better prevention of the spread of dangerous epidemic diseases
1 0.510391216300243 whereas it is expedient to provide for the better prevention of the spread of dangerous epidemic disease it is hereby enacted as follows    short title and extent 1 this act may be called the epidemic diseases act 1897
2 0.5130002770383422 2 2 it extends to the whole of india 3 4   5     definitions in this unless the context otherwise requires    6 a act of violence includes any of the following acts committed by any person against a healthcare service personnel serving during an epidemic which causes or may cause   i harassment impacting the living or working conditions of such healthcare service personnel and preventing him from discharging his duties ii harm injury hurt intimidation or danger to the life of such healthcare service personnel either within the premises of a clinical establishment or otherwise iii obstruction or hindrance to such healthcare

In [ ]:
print("Question: ", question)
print("Answer: ", cleaned_sentences_with_stopwords[index])

Question:  what does composition of certain offences
Answer:  protection to persons acting under act no suit or other legal proceeding shall lie against any person for anything done or in good faith intended to be done under this act
